### **Curriculum Temperature for Knowledge Distillation**

#### **Abstract**
- **前人都没有注意到蒸馏学习中 *temperature* 的重要性**，将其通过网格搜索作为超参数。
- **温度系数 控制两个分布之间的差异，可以确定蒸馏任务的难度**
- *student* 模型越来越聪明，将 *temperature* 设为定值，显然不合理。
- 本文提出了 *Curriculum Temperature for Knowledge Distillation (CTKD)* 算法，通过学生模型的表现来动态调整 *temperature* 的值。**几乎没有增加计算成本。**
- 具体而言，本文设计一个 从简单到复杂的 curriculum，**以一种对抗的方式来提升蒸馏的温度（难度）。**
- 数据集：*CIFAR-100*、*ImageNet-2012* 和 *MS-COCO*

--------------------------------
#### **Introduction**

- *[Meta Knowledge Distillation (2022)](https://arxiv.org/abs/2202.07940)* (MKD) 通过元学习学习合适的温度，**但是它需要一个额外的验证机来训练温度模块。并且它主要适应于强数据增强的ViT，但大部分的蒸馏方法都是普通的数据增强，强行使用反而会使得效果下降。**
- *[Learning From Multiple Experts: Self-paced Knowledge Distillation for Long-tailed Classification (2020)](https://arxiv.org/abs/2001.01536)* (LFME) 采用经典的课程策略，**提出使用简单到难的序列排序的样本逐步训练 student**
- [Knowledge Distillation via Route Constrained Optimization (2019)](https://arxiv.org/abs/1904.09149) (RCO) 提出利用 teacher 中间状态序列作为课程，逐步指导 student 的学习。

> 这都需要复杂的课程设计，而且需要额外的计算成本。

- 本文将通过折腾 temperature 来增加 student 的学习难度，进而提高蒸馏性能。
- **具体来说，动态调整温度，从而提高学习的难度。一言以蔽之，反向梯度，最大化蒸馏损失，从而使得 student 更难学习。**
- [Unsupervised Domain Adaptation by Backpropagation (2015)](https://arxiv.org/abs/1409.7495) 指出，该操作可以很容易地**由非参数梯度反转层实现，以反转温度的梯度**，这几乎不引入额外的计算预算
- **本文提出了可学习温度的两个（全局和局部）版本**，分别是 Global-T 和 Instance-T。
- **CTKD 可以与任何蒸馏学习方法一起使用**

---------------------------------------
#### **Related Work**
- [Curriculum Learning](https://readpaper.com/paper/623410325568061440) 提出于2009年
- 课程学习(Wang, Chen, and Zhu 2021)是一种通过**组织学习任务的顺序和增量增加学习难度**来训练网络的方法(Morerio et al. 2017; Caubri`ere et al. 2019)
- 课程 Dropout (Morerio et al. 2017) 动态增加 dropout 比率以提高模型的泛化能力
- PG-GANs (Karras et al. 2017)学习从低分辨率到高分辨率依次生成图像，同时生成 generator 和 discriminator
- **在知识蒸馏中，各种工作（[Xiang、Ding 和 Han 2020](https://link.springer.com/chapter/10.1007/978-3-030-58558-7_15); [Zhao et al. 2021](https://arxiv.org/abs/2106.10885)）采用课程学习策略来训练学生模型。**

现在的知识蒸馏主要分为三种：
- logit-based (Chen et al. 2020; Li et al. 2020b; Zhao et al. 2022)
- representation-based (Yim et al. 2017; Chen et al. 2021)
- relationship-based (Park et al. 2019; Peng et al. 2019)

--------------------------------------------------
#### **Method**

这里有一个 $\theta_{stu}$ 和 $\theta_{temp}$ 的 mini-max game，可以**应用交替优化**的思想。

\begin{equation}
\begin{split}
&\underset{\theta_{stu}}{\min} \quad \underset{\theta_{temp}}{\max} \quad \mathcal{L} (\theta_{stu}, \theta_{temp}) \\
&= \underset{\theta_{stu}}{\min} \quad \underset{\theta_{temp}}{\max} \quad \sum_{x \in D} \alpha_1 \mathcal{L}_{task} (f^s(x; \theta_{stu}), y) \\
&\quad + \alpha_2 \mathcal{L}_{kd} (f^t(x; \theta_{tea}), f^s(x; \theta_{stu}), \theta_{temp})
\end{split}
\end{equation}

$$\hat{\theta}_{\mathrm{stu}} = arg \quad \underset{\theta_{stu}}{\min} \quad \mathcal{L} (\theta_{stu}, \hat{\theta}_{\mathrm{temp}}) \tag{2}$$

$$\hat{\theta}_{\mathrm{temp}} = arg \quad \underset{\theta_{temp}}{\max} \quad \mathcal{L} (\hat{\theta}_{\mathrm{stu}}, \theta_{temp}) \tag{3}$$


$$\theta_{stu} \leftarrow \theta_{stu} - \mu \frac{\partial \mathcal{L}}{\partial \theta_{stu}} \tag{4}$$

$$\theta_{temp} \leftarrow \theta_{temp} + \mu \frac{\partial \mathcal{L}}{\partial \theta_{temp}} \tag{5}$$

> $\mu$ 是学习率

在实践中，本文实现了上述对抗过程（即 Eqn.(5)) **由非参数梯度反转层 (GRL) (Ganin and Lempitsky 2015)**。GRL插入softmax层和可学习温度模块之间，如图1(a)所示。

![](../img/Figure%201_An%20overview%20of%20our%20proposed%20Curriculum%20Temperature%20for%20Knowledge%20Distillation%20(CTKD)..png)

**为了实现 curriculum learning 的由易到难，我们通过 $\lambda$ 来对 $\mathcal{L}$ 进行直接缩放。**

$$ \theta_{temp} \leftarrow \theta_{temp} + \mu \frac{\partial (\lambda \mathcal{L})}{\partial \theta_{temp}} \tag{6}$$

$\lambda$ 的初始值为 0，且满足如下两个条件：
$$\mathcal{L}_{kd}(\tau_{n+1}) \geq \mathcal{L}_{kd}(\tau_n) \tag{7}$$
$$\lambda_{n+1} \geq \lambda_n \tag{8}$$

在 $E_n$ epoch 的训练时，作者以余弦调度逐渐增加 λ，如下所示

$$\lambda_n = \lambda_{min} + \frac{1}{2}(\lambda_{max} - \lambda_{min})(1 + \cos(1 + \frac{min(E_n,E_{loops})}{E_{loops}})\pi) \tag{9}$$

**其中 $\lambda_{max}$ 和 $\lambda_{min}$ 是 $\lambda$ 的范围。$E_{loops}$ 是逐渐改变难度尺度 $\lambda$ 的超参数。在本文中，作者默认将 $\lambda_{max}$、$\lambda_{min}$ 和 $E_{loops}$ 设置为 1、0 和 10。这个 curriculum 表明，在训练 10 个 epoch 期间参数 $\lambda$ 从 0 增加到 1，并保持 1 直到结束**

![](../img/Figure%202_The%20illustrations%20of%20global%20and%20instance-wise%20temperature%20modules..png)

- **Global-T**：所有样本一个 $\tau$
- **Instance-T**：每个样本一个 $\tau$

---------------------------------------------
#### **总结**
- 关注 temperature 的重要性，并提出了一个简单的方法来动态调整 temperature，从而提高蒸馏的性能。
- 引入了一个 curriculum 来逐步提高学习难度，从而提高蒸馏的性能。
- 引入了 梯度反转层（GRL） 来实现对抗训练，从而提高蒸馏的性能。
- 引入了两种温度模块（Global-T 和 Instance-T），从而提高蒸馏的性能。